In [ ]:
# importing

from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.core.display import display, HTML

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import folium
import plotly.graph_objects as go
import seaborn as sns
import ipywidgets as widgets

In [ ]:
%%html
<script>
    // AUTORUN ALL CELLS ON NOTEBOOK-LOAD!
    require(
        ['base/js/namespace', 'jquery'], 
        function(jupyter, $) {
            $(jupyter.events).on("kernel_ready.Kernel", function () {
                console.log("Auto-running all cells-below...");
                jupyter.actions.call('jupyter-notebook:run-all-cells-below');
                jupyter.actions.call('jupyter-notebook:save-notebook');
            });
        }
    );
</script>

In [ ]:
from IPython.display import HTML
HTML('''<script>
  function code_toggle() {
    if (code_shown){
      $('div.input').hide('500');
      $('#toggleButton').val('Show Code')
    } else {
      $('div.input').show('500');
      $('#toggleButton').val('Hide Code')
    }
    code_shown = !code_shown
  }
  $( document ).ready(function(){
    code_shown=false;
    $('div.input').hide()
  });
</script>
<form action="javascript:code_toggle()"><input type="submit" id="toggleButton" value="Show Code"></form>''')

# Interactive Dashboard: Covid-19 

In [ ]:
# load data
death_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
confirmed_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
recovered_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
country_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/web-data/data/cases_country.csv')

## Confirmed Cases

[full confirmed list](confirmed_cases.ipynb)

In [ ]:
confirmed_df.head(5)

## Death Cases

[full death list](death_cases.ipynb)

In [ ]:
death_df.head(5)

## Recovered Cases

[full recovered list](recovered_cases.ipynb)

In [ ]:
recovered_df.head(5)

## Cases of Each Country

[full country list](country_cases.ipynb)

In [ ]:
country_df.head(5)

In [ ]:
# data cleaning - renaming
country_df.columns = map(str.lower, country_df.columns)
recovered_df.columns = map(str.lower,recovered_df.columns)
death_df.columns = map(str.lower, death_df.columns)
confirmed_df.columns = map(str.lower,confirmed_df.columns)

In [ ]:
confirmed_df = confirmed_df.rename(columns = {'province/state':'state','country/region':'country'})
recovered_df = recovered_df.rename(columns = {'province/state':'state','country/region':'country'})
death_df = death_df.rename(columns = {'province/state':'state','country/region':'country'})
country_df = country_df.rename(columns = {'country_region':'country'})
                                            
                                            

In [ ]:
# total number of confirmed, death and recovered cases
confirmed_total = int(country_df['confirmed'].sum())
deaths_total = int(country_df['deaths'].sum())
recovered_total = int(country_df['recovered'].sum())
active_total = int(country_df['active'].sum())

In [ ]:
# displaying the total stats

display(HTML("<div style = 'background-color: #5499C7; padding: 55px; border-style = 'solid' '>" +
             "<span style='color: #fff; font-size:35px;font-family:serif'> Confirmed: "  + str(confirmed_total) +"</span>" +
             "<span style='color: #E74C3C; font-size:35px;margin-left:20px;font-family:serif'> Deaths: " + str(deaths_total) + "</span>"+
             "<span style='color: #2ECC71; font-size:35px; margin-left:20px;font-family:serif'> Recovered: " + str(recovered_total) + "</span>"+
             "</div>")
       )


## Global Map: The Spread of Covid-19
#### Hover your mouse on a circle to that country's case amounts

In [ ]:
world_map = folium.Map(location = [11,0], tiles = 'cartodbpositron', zoom_start = 2,
                     max_zoom = 6, min_zoom = 2)

for i in range(len(confirmed_df)):
    folium.Circle(
    location = [confirmed_df.iloc[i]['lat'], confirmed_df.iloc[i]['long']],
    fill = True,
    radius = (int((np.log(confirmed_df.iloc[i,-1]+1.00001)))+0.2) * 50000,
    fill_color = '#3498DB',
    color = "red",
    tooltip = "<div style='margin: 0; background-color: black; color: white;'>"+
                    "<h4 style='text-align:center;font-weight: bold'>"+confirmed_df.iloc[i]['country'] + "</h4>"
                    "<hr style='margin:10px;color: white;'>"+
                    "<ul style='color: white;;list-style-type:circle;align-item:left;padding-left:20px;padding-right:20px'>"+
                        "<li>Confirmed: "+str(confirmed_df.iloc[i,-1])+"</li>"+
                        "<li>Deaths:   "+str(death_df.iloc[i,-1])+"</li>"+
                        "<li>Death Rate: "+ str(np.round(death_df.iloc[i,-1]/(confirmed_df.iloc[i,-1]+1.00001)*100,2))+ "</li>"+
                    "</ul></div>",
    ).add_to(world_map)

world_map


<img src="Covid_19Fremont (1).gif" width="850" align="center">

In [ ]:
sorted_country_df = country_df.sort_values('confirmed', ascending = False).head(10)

## View the details of the worst affected countries
#### Enter the number of countries that you want data for

In [ ]:
# sorting the values by confirmed descednding order
# country_df.sort_values('confirmed', ascending= False).head(10).style.background_gradient(cmap='copper')
fig = go.FigureWidget( layout=go.Layout() )
def highlight_col(x):
    r = 'background-color: red'
    g = 'background-color: grey'
    gr = 'background-color: green'
    df1 = pd.DataFrame('', index=x.index, columns=x.columns)
    df1.iloc[:, 4] = g
    df1.iloc[:, 5] = r
    df1.iloc[:, 6] = gr
    
    return df1

def show_latest_cases(number):
    number = int(number)
    return country_df.sort_values('confirmed', ascending= False).head(number).style.apply(highlight_col, axis=None)

interact(show_latest_cases, number='10')

ipywLayout = widgets.Layout(border='solid 2px green')
ipywLayout.display='none' # uncomment this, run cell again - then the graph/figure disappears
widgets.VBox([fig], layout=ipywLayout)

#### Adjust the slider to the number of countries you want data for

In [ ]:
## plotting the worst hit countries

def bubble_chart(n):
    fig = px.scatter(sorted_country_df.head(n), x="country", y="confirmed", size="confirmed", color="country",
               hover_name="country", size_max=60)
    fig.update_layout(
    title=str(n) +" Worst hit countries",
    xaxis_title="Countries",
    yaxis_title="Confirmed Cases",
    width = 1100
    )
    fig.show();

interact(bubble_chart, n=10);

ipywLayout = widgets.Layout(border='solid 2px green')
ipywLayout.display='none'
widgets.VBox([fig], layout=ipywLayout)

## View the details of a specific country or the world
#### Enter the country name and capitalize the first letter

In [ ]:
import plotly.graph_objects as go

def plot_cases_for_country(country):
    labels = ['confirmed', 'deaths']
    colors = ['blue','red']
    mode_size = [6,8]
    line_size = [4,5]

    df_list = [confirmed_df,death_df]

    fig = go.Figure()

    for i, df in enumerate(df_list):
        if country == "World" or country == 'world':
            x_data = np.array(list(df.iloc[:,5:].columns))
            y_data = np.sum(np.asarray(df.iloc[:,5:]), axis = 0)
            
        else:
            x_data = np.array(list(df.iloc[:,5:].columns))
            y_data = np.sum(np.asarray(df[df['country']==country].iloc[:,5:]),axis = 0)
        
        fig.add_trace(go.Scatter(x = x_data, y = y_data, mode = 'lines+markers',
                                name = labels[i],
                                line = dict(color = colors[i], width = line_size[i]),
                                connectgaps = True,
                                text = "Total " + str(labels[i]) + ': ' + str(y_data[-1])
                                ))
    fig.show()
        
#plot_cases_for_country('Brazil')

interact(plot_cases_for_country, country='World');
        

## Bar Charts: Top Ten Worst Affected Countries

In [ ]:
px.bar(
    sorted_country_df.head(10),
    x = "country",
    y = "confirmed",
    title= "Confirmed Cases", # the axis names
    color_discrete_sequence=["lightgrey"], 
    height=500,
    width=800
)

In [ ]:
px.bar(
    sorted_country_df.head(10),
    x = "country",
    y = "deaths",
    title= "Death Cases", # the axis names
    color_discrete_sequence=["pink"], 
    height=500,
    width=800
)

In [ ]:
px.bar(
    sorted_country_df.head(10),
    x = "country",
    y = "recovered",
    title= "Recovered Cases", # the axis names
    color_discrete_sequence=["lightgreen"], 
    height=500,
    width=800
)